### Distance of Sentences

In [1]:
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
vectorizer = CountVectorizer(min_df = 1)

In [3]:
contents = ['상처받은 아이들은 너무 일찍 커버려',
           '내가 상처받은 거 아는 사람 불편해',
           '잘 사는 사람들은 좋은 사람 되기 쉬워',
           '아무 일도 아니야 괜찮아']


In [5]:
from konlpy.tag import Okt
t = Okt()

### 형태소 분석

In [6]:
contents_tokens = [t.morphs(row) for row in contents]
contents_tokens

[['상처', '받은', '아이', '들', '은', '너무', '일찍', '커버', '려'],
 ['내', '가', '상처', '받은', '거', '아는', '사람', '불편해'],
 ['잘', '사는', '사람', '들', '은', '좋은', '사람', '되기', '쉬워'],
 ['아무', '일도', '아니야', '괜찮아']]

### 형태소 분석된 결과를 다시 하나의 문장씩으로 합친다

In [7]:
contents_for_vectorize = []

for content in contents_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
        
    contents_for_vectorize.append(sentence)
    
contents_for_vectorize

[' 상처 받은 아이 들 은 너무 일찍 커버 려',
 ' 내 가 상처 받은 거 아는 사람 불편해',
 ' 잘 사는 사람 들 은 좋은 사람 되기 쉬워',
 ' 아무 일도 아니야 괜찮아']

### 벡터라이즈

In [8]:
X = vectorizer.fit_transform(contents_for_vectorize)
X

<4x17 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

### 4개의 문장에 전체 말뭉치의 단어가 17개다

In [9]:
num_samples, num_features = X.shape
num_samples, num_features

(4, 17)

In [10]:
vectorizer.get_feature_names()

['괜찮아',
 '너무',
 '되기',
 '받은',
 '불편해',
 '사는',
 '사람',
 '상처',
 '쉬워',
 '아는',
 '아니야',
 '아무',
 '아이',
 '일도',
 '일찍',
 '좋은',
 '커버']

In [11]:
X.toarray().transpose()

array([[0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 2, 0],
       [1, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0]], dtype=int64)

### 테스트용 문장

In [12]:
new_post = ['상처받기 싫어 괜찮아']
new_post_tokens = [t.morphs(row) for row in new_post]

new_post_for_vectorize  = []

for content in new_post_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
        
    
    new_post_for_vectorize.append(sentence)
    
new_post_for_vectorize

[' 상처 받기 싫어 괜찮아']

In [13]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)


### 벡터로 표현

In [14]:
new_post_vec.toarray()

array([[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

### 유클리드 거리

In [15]:
import scipy as sp

def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

### 결과

In [16]:
dist = [dist_raw(each, new_post_vec) for each in X]
dist

[2.449489742783178, 2.23606797749979, 3.1622776601683795, 2.0]

In [18]:
print("Best post is ", dist.index(min(dist)), ", dist = ", min(dist))
print("test post is -->", new_post)
print("best dist post is -->", contents[dist.index(min(dist))])

Best post is  3 , dist =  2.0
test post is --> ['상처받기 싫어 괜찮아']
best dist post is --> 아무 일도 아니야 괜찮아


### 유사도- 벡터사이의 거리

In [19]:
for i in range(0, len(contents)):
    print(X.getrow(i).toarray())
    

print('----------------------')
print(new_post_vec.toarray())

[[0 1 0 1 0 0 0 1 0 0 0 0 1 0 1 0 1]]
[[0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0]]
[[0 0 1 0 0 1 2 0 1 0 0 0 0 0 0 1 0]]
[[1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0]]
----------------------
[[1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]]


### TFIDF
    - 한 문서에서 많이 등장한 단어에 가중치를 부여
    - 전체 문서에서 많이 나타나는 단어는 중요하지 않게 

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1, decode_error='ignore')

In [22]:
X = vectorizer.fit_transform(contents_for_vectorize)
num_samples, num_features = X.shape
num_samples, num_features

(4, 17)

In [23]:
X.toarray().transpose()

array([[0.        , 0.        , 0.        , 0.5       ],
       [0.43671931, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.34431452, 0.40104275, 0.        , 0.        ],
       [0.        , 0.50867187, 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.        , 0.40104275, 0.6191303 , 0.        ],
       [0.34431452, 0.40104275, 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.        , 0.50867187, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.5       ],
       [0.        , 0.        , 0.        , 0.5       ],
       [0.43671931, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.5       ],
       [0.43671931, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.43671931, 0.        , 0.        , 0.        ]])

### 테스트 문장 적용

In [24]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[0.78528828, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.6191303 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

In [26]:
def dist_norm(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())
    
    delta = v1_normalized - v2_normalized
    
    return sp.linalg.norm(delta.toarray())

### 결과

In [27]:
dist = [dist_norm(each, new_post_vec) for each in X]

print("Best post is ", dist.index(min(dist)), ", dist = ", min(dist))
print("test post is -->", new_post)
print("best dist post is -->", contents[dist.index(min(dist))])

Best post is  3 , dist =  1.1021396119773588
test post is --> ['상처받기 싫어 괜찮아']
best dist post is --> 아무 일도 아니야 괜찮아


In [29]:
import urllib.request
import json
import datetime

### 네이버 API를 사용

In [30]:
def gen_search_url(api_node, search_text, start_num, disp_num):
    base = 'https://openapi.naver.com/v1/search'
    node = "/" + api_node + ".json"
    param_query = "?query=" + urllib.parse.quote(search_text)
    param_start = '&start=' + str(start_num)
    param_disp = '&display=' + str(disp_num)
    
    return base + node + param_query + param_start + param_disp

In [31]:
import json
import datetime

def get_result_onpage(url):
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    
    response = urllib.request.urlopen(request)
    
    print("[%s] Url Request Success" % datetime.datetime.now())
    
    return json.loads(response.read().decode('utf-8'))

### 문장 수집(파이썬에 관련된 검색)

In [58]:
client_id = '****'
client_secret ='*****'

url = gen_search_url('kin', '파이썬', 1, 100)
one_result = get_result_onpage(url)
one_result

[2020-04-17 17:30:10.719269] Url Request Success


{'lastBuildDate': 'Fri, 17 Apr 2020 17:30:10 +0900',
 'total': 128679,
 'start': 1,
 'display': 100,
 'items': [{'title': '<b>파이썬</b>만 공부해도 취업 가능할까요?',
   'link': 'https://kin.naver.com/qna/detail.nhn?d1id=4&dirId=40608&docId=349586757&qb=7YyM7J207I2s&enc=utf8&section=kin.qna&rank=1&search_sort=0&spq=0',
   'description': '... 때마다 <b>파이썬</b>으로 백준 문제를 풀고있고 Mysql과... 제가 알기로는 학교에서 하는 프로그래밍 대회는 <b>파이썬</b>이... 정말 <b>파이썬</b>만 해서 실력으로 취업할 가능성도 있을까요?... <b>파이썬</b> 공부에 관심을 갖고 계신 상황인데요. 기본적으로... '},
  {'title': '<b>파이썬</b> 공부 <b>파이썬</b> 코딩',
   'link': 'https://kin.naver.com/qna/detail.nhn?d1id=1&dirId=10402&docId=347839941&qb=7YyM7J207I2s&enc=utf8&section=kin.qna&rank=2&search_sort=0&spq=0',
   'description': '이제 <b>파이썬</b>으로 실습을 하려는 대학생입니다. <b>파이썬</b>을 혼자 공부하다보니 책에서 모듈을 공부하는데... 말인데 <b>파이썬</b> 모듈을 좀 더 공부할 수 있는 곳이나 자료를 어디서 받을지 알 수 있을가요?  <b>파이썬</b>에서... '},
  {'title': '<b>파이썬</b>학원 빨리배울곳?',
   'link': 'https://kin.naver.com/qna/detail.nhn?d1id=4&dirId=40608&docId=353194881&qb=7YyM7J207I2s&enc=

In [59]:
one_result['items'][0]['description']

'... 때마다 <b>파이썬</b>으로 백준 문제를 풀고있고 Mysql과... 제가 알기로는 학교에서 하는 프로그래밍 대회는 <b>파이썬</b>이... 정말 <b>파이썬</b>만 해서 실력으로 취업할 가능성도 있을까요?... <b>파이썬</b> 공부에 관심을 갖고 계신 상황인데요. 기본적으로... '

### 태그 제거기

In [60]:
def delete_tag(input_str):
    input_str = input_str.replace("<b>", "")
    input_str = input_str.replace("</b>", "")
    
    return input_str

In [61]:
def get_description(pages):
    contents = []
    for sentences in pages['items']:
        contents.append(delete_tag(sentences['description']))
        
    return contents

### 컨텐츠를 list로 변환

In [62]:
contents = get_description(one_result)
contents

['... 때마다 파이썬으로 백준 문제를 풀고있고 Mysql과... 제가 알기로는 학교에서 하는 프로그래밍 대회는 파이썬이... 정말 파이썬만 해서 실력으로 취업할 가능성도 있을까요?... 파이썬 공부에 관심을 갖고 계신 상황인데요. 기본적으로... ',
 '이제 파이썬으로 실습을 하려는 대학생입니다. 파이썬을 혼자 공부하다보니 책에서 모듈을 공부하는데... 말인데 파이썬 모듈을 좀 더 공부할 수 있는 곳이나 자료를 어디서 받을지 알 수 있을가요?  파이썬에서... ',
 '... 생각중이라 파이썬학원 다니면서 미리 좀 배워보려고 파이썬학원 알아보는중인데요. 집 주변에서 파이썬학원... 이왕 파이썬학원 다니는거 전문적인곳으로 잘 알아보고 다니고 싶은데 커리큘럼 다양하고 시설 괜찮은... ',
 '파이썬 공부를 하려고 하는데 교재는 어떤게 좋으며, 그냥 검색해서 나오는 파이썬 아무거나 깔면 상관 없는건가요?? 부모님이 자꾸 파이썬 어디서 구매하는지... 파이썬은 무로입니다. 파이선 자체 설치는 한곳 뿐 입니다.... ',
 '라즈베리파이 파이썬 2 추가 설치 어떻게 하나요? 라즈비안 최신버전입니다 라즈비안에는 파이썬 2.7 과 3.4 버전이 기본적으로 설치되어 있습니다. 실행시 python 으로 실행하면 2.7 버전대로 실행되고 python3 로 실행하면... ',
 '... 인터넷 검색해보니깐 파이썬을 먼저 배우는게 좋다고 하더라구요 파이썬학원 검색해보는데 정보가 너무 많아서... 파이썬학원 중 가장 좋은곳은 어디인가요? 비전공자이고, 파이썬에 파도 모르거든요.... 파이썬학원 선택하는데... ',
 '파이썬으로 프로그래밍 처음 입문하려고합니다. 프로그래밍 입문하려면 파이썬이 좋다는 이야기를 많이... 파이썬을 공부하신다고 하셨는데요~ 일단 냉정하게... 추가적으로, 만약 주위에 파이썬 관련하여 전문가가 있다면, 그... ',
 '안녕하세요 파이썬 공부를 시작으로 프로그래밍 언어 공부를 하려는 평범한... 선후배들이 파이썬 언어는 바워 놓으면 도움된다해서 파이

### 형태소 분석

In [63]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt

t = Okt()
vectorizer = CountVectorizer(min_df = 1)


In [64]:
contents_tokens = [t.morphs(row) for row in contents ]
contents_tokens

[['...',
  '때',
  '마다',
  '파이썬',
  '으로',
  '백',
  '준',
  '문제',
  '를',
  '풀고있고',
  'Mysql',
  '과',
  '...',
  '제',
  '가',
  '알기',
  '로는',
  '학교',
  '에서',
  '하는',
  '프로그래밍',
  '대회',
  '는',
  '파이썬',
  '이',
  '...',
  '정말',
  '파이썬',
  '만',
  '해서',
  '실력',
  '으로',
  '취업',
  '할',
  '가능성',
  '도',
  '있을까요',
  '?...',
  '파이썬',
  '공부',
  '에',
  '관심',
  '을',
  '갖고',
  '계신',
  '상황',
  '인데',
  '요',
  '.',
  '기본',
  '적',
  '으로',
  '...'],
 ['이제',
  '파이썬',
  '으로',
  '실습',
  '을',
  '하려는',
  '대학생',
  '입니다',
  '.',
  '파이썬',
  '을',
  '혼자',
  '공부',
  '하다',
  '보니',
  '책',
  '에서',
  '모듈',
  '을',
  '공부',
  '하는데',
  '...',
  '말',
  '인데',
  '파이썬',
  '모듈',
  '을',
  '좀',
  '더',
  '공부',
  '할',
  '수',
  '있는',
  '곳',
  '이나',
  '자료',
  '를',
  '어디서',
  '받을지',
  '알',
  '수',
  '있을가요',
  '?',
  '파이썬',
  '에서',
  '...'],
 ['...',
  '생각',
  '중',
  '이라',
  '파이썬',
  '학원',
  '다니면서',
  '미리',
  '좀',
  '배워',
  '보려고',
  '파이썬',
  '학원',
  '알아보는',
  '중',
  '인데',
  '요',
  '.',
  '집',
  '주변',
  '에서',
  '파이썬',
  '학원',
  '...',
  '이왕',


### vectorize 후 다시 합치기

In [65]:
contents_for_vectorize = []

for content in contents_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
        
    contents_for_vectorize.append(sentence)
    
contents_for_vectorize

[' ... 때 마다 파이썬 으로 백 준 문제 를 풀고있고 Mysql 과 ... 제 가 알기 로는 학교 에서 하는 프로그래밍 대회 는 파이썬 이 ... 정말 파이썬 만 해서 실력 으로 취업 할 가능성 도 있을까요 ?... 파이썬 공부 에 관심 을 갖고 계신 상황 인데 요 . 기본 적 으로 ...',
 ' 이제 파이썬 으로 실습 을 하려는 대학생 입니다 . 파이썬 을 혼자 공부 하다 보니 책 에서 모듈 을 공부 하는데 ... 말 인데 파이썬 모듈 을 좀 더 공부 할 수 있는 곳 이나 자료 를 어디서 받을지 알 수 있을가요 ? 파이썬 에서 ...',
 ' ... 생각 중 이라 파이썬 학원 다니면서 미리 좀 배워 보려고 파이썬 학원 알아보는 중 인데 요 . 집 주변 에서 파이썬 학원 ... 이왕 파이썬 학원 다니는거 전문 적 인 곳 으로 잘 알아보고 다니고 싶은데 커리큘럼 다양하고 시설 괜찮은 ...',
 ' 파이썬 공부 를 하려고 하는데 교재 는 어떤게 좋으며 , 그냥 검색 해서 나오는 파이썬 아무 거나 깔면 상관 없는건 가요 ?? 부모님 이 자꾸 파이썬 어디서 구매 하는지 ... 파이썬 은 무 로 입니다 . 파이선 자체 설치는 한 곳 뿐 입니다 ....',
 ' 라즈베리파이 파이썬 2 추가 설치 어떻게 하나요 ? 라즈 비안 최신 버전 입니다 라즈 비안에는 파이썬 2.7 과 3.4 버전 이 기본 적 으로 설치 되어 있습니다 . 실행 시 python 으로 실행 하면 2.7 버전 대로 실행 되고 python 3 로 실행 하면 ...',
 ' ... 인터넷 검색 해보니깐 파이썬 을 먼저 배우는게 좋다고 하더라구요 파이썬 학원 검색 해보는데 정보 가 너무 많아서 ... 파이썬 학원 중 가장 좋은 곳 은 어디 인가요 ? 비 전공자 이고 , 파이썬 에 파도 모르거든요 .... 파이썬 학원 선택 하는데 ...',
 ' 파이썬 으로 프로그래밍 처음 입문 하려고 합니다 . 프로그래밍 입문 하려면 파이썬 이 좋다는 이야기 를 많이 ... 파이썬 을 공부 하신다고 하셨는데요 ~ 일단

### 결과

In [66]:
X = vectorizer.fit_transform(contents_for_vectorize)
X

<100x1013 sparse matrix of type '<class 'numpy.int64'>'
	with 2344 stored elements in Compressed Sparse Row format>

### 100개의 문장안에 1013개의 말뭉치

In [67]:
num_samples, num_features = X.shape
num_samples, num_features

(100, 1013)

### 벡터라이즈 결과

In [68]:
vectorizer.get_feature_names()

['032',
 '06',
 '10',
 '100',
 '15',
 '1회',
 '20',
 '2013',
 '2019',
 '24',
 '2년',
 '300',
 '40',
 '60',
 '71',
 'aa',
 'amp',
 'append',
 'array',
 'as',
 'cms2580',
 'co',
 'coding',
 'com',
 'db',
 'dynamic',
 'edu',
 'edueroom',
 'encoding',
 'executable',
 'file',
 'following',
 'for',
 'format',
 'forward',
 'goto',
 'gt',
 'html',
 'http',
 'https',
 'idle',
 'if',
 'import',
 'in',
 'input',
 'installer',
 'int',
 'it',
 'join',
 'kg',
 'kin',
 'kr',
 'limhg999',
 'list',
 'll',
 'lst',
 'lt',
 'map',
 'mysql',
 'naver',
 'next',
 'open',
 'oupput',
 'ozsrank',
 'pen',
 'pip',
 'pop',
 'previous',
 'print',
 'pused',
 'push',
 'py',
 'python',
 'quot',
 'random',
 'range',
 'ricnn',
 'sbs',
 'sbsdaejeon',
 'scode',
 'stack',
 'super',
 'the',
 'this',
 'tistory',
 'top',
 'turtle',
 'uses',
 'utf',
 'while',
 'yes',
 'ㅜㅜ',
 'ㅠㅠ',
 '가능성',
 '가능하다고',
 '가능하다면',
 '가능한',
 '가능할까',
 '가르쳐주는',
 '가변',
 '가서',
 '가실',
 '가야',
 '가요',
 '가장',
 '가지',
 '각도',
 '간결해서',
 '간단하게',
 '간단하며',
 '갈아탄지',
 '강

### 유사도 측정할 테스트 문장

In [69]:
new_post = ["파이썬을 배우는데 좋은 방법이 어떤 것인지 추천해주세요"]
new_post_tokens = [t.morphs(row) for row in new_post]

new_post_for_vectorize = []

for content in new_post_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
        
    new_post_for_vectorize.append(sentence)
    
new_post_for_vectorize

[' 파이썬 을 배우는데 좋은 방법 이 어떤 것 인지 추천 해주세요']

In [70]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

### 유클리드 거리

In [71]:
import scipy as sp

def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

In [72]:
dist = [dist_raw(each, new_post_vec) for each in  X]
dist

[6.855654600401044,
 6.928203230275509,
 7.14142842854285,
 6.48074069840786,
 8.0,
 7.211102550927978,
 6.708203932499369,
 7.280109889280518,
 6.082762530298219,
 6.6332495807108,
 6.48074069840786,
 5.830951894845301,
 7.0710678118654755,
 6.782329983125268,
 6.928203230275509,
 6.708203932499369,
 6.6332495807108,
 6.855654600401044,
 7.14142842854285,
 6.6332495807108,
 6.557438524302,
 6.6332495807108,
 6.0,
 7.0710678118654755,
 6.928203230275509,
 7.483314773547883,
 6.6332495807108,
 7.0710678118654755,
 7.14142842854285,
 6.928203230275509,
 6.0,
 7.3484692283495345,
 6.244997998398398,
 6.6332495807108,
 5.744562646538029,
 7.0,
 7.416198487095663,
 6.4031242374328485,
 6.557438524302,
 6.708203932499369,
 8.366600265340756,
 6.6332495807108,
 6.782329983125268,
 6.4031242374328485,
 6.782329983125268,
 6.4031242374328485,
 5.744562646538029,
 6.782329983125268,
 6.708203932499369,
 5.656854249492381,
 7.681145747868608,
 7.615773105863909,
 6.708203932499369,
 6.70820393249

### 테스트결과

In [73]:
print("Best post is ", dist.index(min(dist)), ", dist = ", min(dist))
print("test post is -->", new_post)
print("best dist post is -->", contents[dist.index(min(dist))])

Best post is  69 , dist =  5.0
test post is --> ['파이썬을 배우는데 좋은 방법이 어떤 것인지 추천해주세요']
best dist post is --> 파이썬 알고리즘 문제 푸는데요 이 코드가 맞는 것 같고 실제로 파이썬 프로그램 돌려보면 잘 나오는데 아니라고 하네요 어떤 걸 바꿔야 하는 건가요? 첫줄에 두개의 수가 입력 된다고 했는데, 두 줄에... 


### 거리를 구하는 다른방법 normalize

In [74]:
def dist_norm(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())
    
    delta = v1_normalized - v2_normalized
    
    return sp.linalg.norm(delta.toarray())

In [75]:
dist = [dist_raw(each, new_post_vec) for each in  X]
print("Best post is ", dist.index(min(dist)), ", dist = ", min(dist))
print("test post is -->", new_post)
print("best dist post is -->", contents[dist.index(min(dist))])

Best post is  69 , dist =  5.0
test post is --> ['파이썬을 배우는데 좋은 방법이 어떤 것인지 추천해주세요']
best dist post is --> 파이썬 알고리즘 문제 푸는데요 이 코드가 맞는 것 같고 실제로 파이썬 프로그램 돌려보면 잘 나오는데 아니라고 하네요 어떤 걸 바꿔야 하는 건가요? 첫줄에 두개의 수가 입력 된다고 했는데, 두 줄에... 


### Tfidf

In [76]:
def tfidf(t, d, D):
    tf = float(d.count(t)) / sum(d.count(w) for w in set(d))
    idf = sp.log( float(len(D))/(len([doc for doc in D if t in doc])) )
    return tf*idf

In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1, decode_error='ignore')


In [78]:
X = vectorizer.fit_transform(contents_for_vectorize)
num_samples, num_features = X.shape
num_samples, num_features

(100, 1013)

In [79]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [80]:
dist = [dist_raw(each, new_post_vec) for each in  X]
print("Best post is ", dist.index(min(dist)), ", dist = ", min(dist))
print("test post is -->", new_post)
print("best dist post is -->", contents[dist.index(min(dist))])

Best post is  63 , dist =  1.1900388593487985
test post is --> ['파이썬을 배우는데 좋은 방법이 어떤 것인지 추천해주세요']
best dist post is --> 프로그래밍 입문할 땐 파이썬이 좋다고 해서 파이썬 독학해보려고 하는데요 어떤 식으로 독학 시작해야할지 감이 안오네요 파이썬 독학하는 방법 추천좀 해주세요 보통 파이썬 독학은 인강으로 많이 하는 편이에요 파이썬 독학... 


In [81]:
client_id = 'uCFS2FeQh0Hd5Ljzztue'
client_secret ='Jew0Xamv_h'
 
url = gen_search_url('book', '파이썬', 1, 100)
one_result = get_result_onpage(url)
one_result

[2020-04-17 17:32:21.215869] Url Request Success


{'lastBuildDate': 'Fri, 17 Apr 2020 17:32:21 +0900',
 'total': 1030,
 'start': 1,
 'display': 100,
 'items': [{'title': '<b>파이썬</b> (비전공자를위한 <b>파이썬</b> 기초 입문서,한 권으로 끝내는)',
   'link': 'http://book.naver.com/bookdb/book_detail.php?bid=14921665',
   'image': 'https://bookthumb-phinf.pstatic.net/cover/149/216/14921665.jpg?type=m1&udate=20190514',
   'author': '김명호',
   'price': '18000',
   'discount': '16200',
   'publisher': '에듀웨이',
   'pubdate': '20190525',
   'isbn': '1186179368 9791186179369',
   'description': '막막했던 <b>파이썬</b>, 이제 한 권으로 기초를 배우는\n<b>파이썬</b> 사용 설명서!전 세계적으로 유명한 구글(GOOGLE)이 자신의 많은 서비스를 <b>파이썬</b>으로 제공하고 있습니다. 이것은 구글의 뛰어난 서비스를 개인의 비즈니스에 쉽게 이용할 수 있다는 의미입니다.  이러한 발전에 맞춰 <b>파이썬</b> 문법의 기초를 쉽게 이해할 수... '},
  {'title': '혼자 공부하는 <b>파이썬</b> (<b>파이썬</b> 최신 버전 반영)',
   'link': 'http://book.naver.com/bookdb/book_detail.php?bid=15028688',
   'image': 'https://bookthumb-phinf.pstatic.net/cover/150/286/15028688.jpg?type=m1&udate=20200321',
   'author': '윤인성',
   'price': '18000',
   'd